<div style="text-align: center; background-color: #ADD8E6; font-family: 'Times New Roman', serif; color: white; padding: 20px; font-size: 30px; font-weight: bold; border-radius: 10px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); transition: all 0.3s ease;">
    THE ANALYSIS OF THE MOBILE PHONE MARKET AT MOBILE CITY STORE
</div>
<style>
    div:hover {
        transform: scale(1.05);
        box-shadow: 0px 10px 20px rgba(0, 0, 0, 0.3);
    }
</style>


---

<div style="text-align: center; background-color: #ADD8E6; font-family: 'Times New Roman', serif; color: white; padding: 25px; font-size: 20px; font-weight: bold; border-radius: 10px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); transition: all 0.3s ease;">
    DATA COLLECTION
</div>
<style>
    div:hover {
        opacity: 0.8;
    }
</style>

---

# Thư viện

Sử dụng Selenium kết hợp với BeautySoup để thu thập dữ liệu

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse
import csv

import pandas as pd

# Lấy source page trang `https://mobilecity.vn/dien-thoai`

In [2]:
# Get current time
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Print the current time
print("Thời gian bắt đầu thu thập:", current_time)

Thời gian bắt đầu thu thập: 2024-12-18 03:29:11


> Thu thập toàn bộ đường dẫn đến mỗi điện thoại bằng cách nhấn xem thêm cho đến khi nào tải được hết toàn bộ điện thoại (không thể nhấn xem thêm nữa).

In [3]:
# Tạo instance của trình duyệt
driver = webdriver.Chrome()

driver.maximize_window()

# Truy cập vào trang web
url = "https://mobilecity.vn/dien-thoai"
driver.get(url)

# Hàm nhấn nút "Xem thêm" nếu có thể
def click_see_more_button():
    try:
        # Tìm nút "Xem thêm điện thoại"
        see_more_button = driver.find_element(By.ID, "product_view_more")
        if see_more_button.is_displayed():
            # Nhấn nút
            see_more_button.click()
            # Đợi một chút để trang tải thêm sản phẩm
            time.sleep(2)
            return True
    except (NoSuchElementException, ElementClickInterceptedException):
        # Không tìm thấy nút hoặc không thể nhấn
        return False
    return False

# Liên tục nhấn nút "Xem thêm" cho đến khi không còn nút nào
while click_see_more_button():
    pass

# Lấy source page sau khi đã nhấn hết các nút "Xem thêm"
page_source = driver.page_source
# Đóng trình duyệt
driver.quit()

# Thu thập đường dẫn đến từng trang điện thoại chi tiết

In [4]:
# Phân tích cú pháp HTML
soup = BeautifulSoup(page_source, "html.parser")

# Tìm tất cả các sản phẩm
product_items = soup.find_all("div", class_="product-item-info")

# Lưu các liên kết vào danh sách
links = []

for item in product_items:
    # Tìm thẻ <a> chứa liên kết đến chi tiết sản phẩm
    link_tag = item.find("a", href=True)
    if link_tag:
        # Lấy giá trị href
        link = link_tag['href']
        links.append(link)

In [5]:
# Get current time
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Print the current time
print("Thời gian kết thúc thu thập:", current_time)

Thời gian kết thúc thu thập: 2024-12-18 03:32:18


# Lưu vào file `links.csv`

In [6]:
df = pd.DataFrame(links, columns=['links'])
df.to_csv("../data/links.csv", index=True)

Dataset này bao gồm 2 thuộc tính:
- `Index`: số thứ tự của đường dẫn đến từng điện thoại trên trang web MobileCity.
- `Links`: đường dẫn tương ứng với từng điện thoại trên trang web MobileCity.